In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist 

In [3]:
def padImageForPatches(img, patch_shape):
    return cv2.copyMakeBorder(src=img, left=0, right=patch_shape[1]*((img.shape[1]+31)//patch_shape[1])-img.shape[1], top=0, bottom=patch_shape[0]*((img.shape[0]+31)//patch_shape[0])-img.shape[0], borderType=cv2.BORDER_REFLECT)

def divideIntoPatches(img, patch_shape):
    img = padImageForPatches(img, patch_shape)
    img_shape = img.shape
    patch_array = np.empty((int((img_shape[0]*img_shape[1])/(patch_shape[0]*patch_shape[1])), patch_shape[0], patch_shape[1], 3), dtype=np.int32)
    for i in range(0, int(img_shape[0]/patch_shape[0])):
        for j in range(0, int(img_shape[1]/patch_shape[1])):
            patch_array[i*int(img_shape[1]/patch_shape[1]) + j] = img[i*patch_shape[0]:i*patch_shape[0]+patch_shape[0], j*patch_shape[1]:j*patch_shape[1]+patch_shape[1], :]

    return patch_array

def getColorHistFeatures(img, n_bins=8):
    img = img//int(256/n_bins)
    hist = np.bincount(img[:, :, 0].ravel(), minlength=8)
    hist = np.append(hist, np.bincount(img[:, :, 1].ravel(), minlength=8))
    hist = np.append(hist, np.bincount(img[:, :, 2].ravel(), minlength=8))
    return hist


def getHistForPatches(img, patch_shape):
    patch_array = divideIntoPatches(img, patch_shape)
    n, h, w, c = patch_array.shape
    hist_array = np.empty((n, 24), dtype=np.int32)
    for i in range(0, n):
        hist_array[i] = getColorHistFeatures(patch_array[i])
    return hist_array

def loadImagesFromDir(dir_path):
    images = {}
    
    for folder in os.listdir(dir_path):
        img_arr = []
        folder_path = dir_path + "/" + folder
        for file in os.listdir(folder_path):
            img_path = folder_path + "/" + file
            img = cv2.imread(img_path)
            if img is not None:
                img_arr.append(img)
        images[folder] = img_arr
    
    return images

def getHistForAllImages(images, patch_shape):
    images_hist_features = {}

    for key, img_arr in images.items():
        hist24_arr = []
        for img in img_arr:
            hist24_arr.append(getHistForPatches(img, patch_shape))
        images_hist_features[key] = hist24_arr
    
    return images_hist_features

In [4]:
train_images = loadImagesFromDir("Group21/Classification/Image_Group21/train")
test_images = loadImagesFromDir("Group21/Classification/Image_Group21/test")

In [5]:
train_images

{'batters_box': [array([[[246, 242, 241],
          [246, 243, 239],
          [245, 242, 237],
          ...,
          [245, 238, 218],
          [245, 238, 218],
          [245, 238, 218]],
  
         [[247, 243, 242],
          [246, 243, 239],
          [244, 241, 236],
          ...,
          [245, 238, 218],
          [245, 238, 218],
          [245, 238, 218]],
  
         [[247, 243, 242],
          [245, 242, 238],
          [243, 240, 235],
          ...,
          [245, 238, 218],
          [245, 238, 218],
          [245, 238, 218]],
  
         ...,
  
         [[ 71, 102, 133],
          [ 70, 101, 132],
          [ 64,  95, 126],
          ...,
          [112, 142, 167],
          [112, 142, 167],
          [101, 131, 156]],
  
         [[ 80, 111, 142],
          [ 75, 106, 137],
          [ 76, 107, 138],
          ...,
          [102, 132, 159],
          [104, 134, 161],
          [103, 133, 160]],
  
         [[ 75, 106, 137],
          [ 71, 102, 133],
         

In [6]:
train_imgs_hist = getHistForAllImages(train_images, patch_shape=(32, 32))
test_imgs_hist = getHistForAllImages(test_images, patch_shape=(32, 32))

In [49]:
train_imgs_hist_flattened = np.concatenate([y for x in train_imgs_hist.values() for y in x])

In [78]:
train_imgs_hist_flattened

array([[  0,   0,   0, ...,  24,  61, 858],
       [  0,   0,   0, ...,  43,  36, 893],
       [  0,   0,   1, ...,  47,  26, 862],
       ...,
       [  0,   0,   0, ..., 542, 240,  44],
       [  0,   0,   8, ..., 370, 224, 108],
       [  0,   2,   6, ..., 332, 342, 170]])

In [175]:
def kmeans(x, k, no_of_iterations):
    idx = np.random.choice(len(x), k, replace=False)
    print(idx)
    #Randomly choosing Centroids 
    centroids = x[idx, :] #Step 1
     
    #finding the distance between centroids and all the data points
    distances = cdist(x, centroids ,'euclidean') #Step 2
     
    #Centroid with the minimum Distance
    points = np.array([np.argmin(i) for i in distances]) #Step 3
    
    for i in range(k):
        points[idx[i]] = i
        
    
    #Repeating the above steps for a defined number of iterations
    #Step 4
    for itr in range(no_of_iterations): 
        centroids = []
        print(f"*** Iteration {itr} ***")
        for idx in range(k):
            #Updating Centroids by taking mean of Cluster it belongs to
            print(x[points==idx])
            temp_cent = x[points==idx].mean(axis=0) 
            centroids.append(temp_cent)
 
        centroids = np.vstack(centroids) #Updated Centroids 
         
        distances = cdist(x, centroids ,'euclidean')
        points = np.array([np.argmin(i) for i in distances])
         
    return centroids

In [176]:
centroids = kmeans(train_imgs_hist_flattened, 32, 100)

[ 68847  70759  69772  50252 132344  19999 149609  32015 131363   9267
  13919 116694   8781 142301  69882  38325  23892 135879  77207 129067
  53964 129969  44736 111774  98624  14861  40406  17863  27447  87040
  85846 135643]
*** Iteration 0 ***
[[   0    0    0 ...   24   61  858]
 [   0    0    0 ...   43   36  893]
 [   0    0    0 ...    7   42  975]
 ...
 [   0    0    0 ...   20   30  908]
 [   0    0    0 ...    0    0 1024]
 [   0    0    0 ...    6    2 1016]]
[[  4 150 294 ...   0   0   0]
 [  8 128 359 ...   0   0   0]
 [  0 146 302 ...  90  48   3]
 ...
 [  6 112 396 ... 144  74  36]
 [  0  58 304 ... 234  94  14]
 [ 18 162 350 ... 122  56  28]]
[[  0   0   0 ...   0 416 608]
 [  0   0   2 ...   6  75 939]
 [  0   0   0 ...  60 162 788]
 ...
 [  0   0   0 ...   0 469 555]
 [  0   0   0 ...  29 186 784]
 [  0   0   0 ...   0 478 546]]
[[  0   0 300 ...   0 323 701]
 [  0   0 254 ...   0 228 796]
 [  0   0  16 ...   0 531 493]
 ...
 [  3  32 413 ... 186 636 164]
 [  1  49 

*** Iteration 2 ***
[[  0   0   0 ...  24  61 858]
 [  0   0   0 ...  43  36 893]
 [  0   0   1 ...  47  26 862]
 ...
 [  0   0   0 ...  20  30 908]
 [  0   0  58 ...  14  15 765]
 [  0   0   6 ...  10  17 786]]
[[  0 174 329 ...  69   0   0]
 [  1  52 334 ... 160  18   0]
 [  0 136 376 ...  64   1   0]
 ...
 [  6 112 396 ... 144  74  36]
 [  0  58 304 ... 234  94  14]
 [ 18 162 350 ... 122  56  28]]
[[   0    0    0 ...    0  416  608]
 [   0    0    0 ...    0 1008   14]
 [   0   34   54 ...   12  878   22]
 ...
 [   0  102  150 ...   14   20  656]
 [   9   95   86 ...   17   28  724]
 [   0   15  151 ...   25   22  726]]
[[  0   0 300 ...   0 323 701]
 [  0   0 254 ...   0 228 796]
 [  0   0 458 ...   0 345 679]
 ...
 [  0   0   0 ... 219 748  54]
 [  1  17 124 ... 504  90   7]
 [  2  13  74 ... 658 113  28]]
[[419 605   0 ...   0   0   0]
 [314 710   0 ...   0   0   0]
 [292 719  12 ...   0   0   0]
 ...
 [533 452  32 ...   0   0   0]
 [631 392   1 ...   0   0   0]
 [678 190  60 ..

*** Iteration 4 ***
[[  0   0   0 ...  24  61 858]
 [  0   0   0 ...  43  36 893]
 [  0   0   1 ...  47  26 862]
 ...
 [  0   0   0 ...  20  30 908]
 [  0   0  58 ...  14  15 765]
 [  0   0   6 ...  10  17 786]]
[[  1  52 334 ... 160  18   0]
 [  0 136 376 ...  64   1   0]
 [ 20 254 446 ...  53  40  10]
 ...
 [ 25 208 385 ... 277 133  19]
 [  0  20 434 ...  96   0   0]
 [ 38 121 315 ... 242 120  58]]
[[   0    0    0 ...    0 1008   14]
 [   0   34   54 ...   12  878   22]
 [   0    0    0 ...   34  551  388]
 ...
 [   0    6   56 ...   44  592  211]
 [   0    0    0 ...    0  571  453]
 [   0    0    0 ...    0  728  296]]
[[  0   0 300 ...   0 323 701]
 [  0   0 254 ...   0 228 796]
 [  0   0 449 ...   0 227 797]
 ...
 [  5  23  92 ... 431 237  43]
 [  0  11  61 ... 393 307  41]
 [  0  11  63 ... 485 228  38]]
[[425 599   0 ...   0   0   0]
 [450 574   0 ...   0   0   0]
 [423 601   0 ...   0   0   0]
 ...
 [533 452  32 ...   0   0   0]
 [631 392   1 ...   0   0   0]
 [678 190  60 ..

*** Iteration 6 ***
[[  0   0   0 ...  24  61 858]
 [  0   0   1 ...  47  26 862]
 [  0   0  14 ...  51  20 762]
 ...
 [  0  15 151 ...  25  22 726]
 [  0   0  58 ...  14  15 765]
 [  0   0   6 ...  10  17 786]]
[[  1  52 334 ... 160  18   0]
 [  0 136 376 ...  64   1   0]
 [ 20 254 446 ...  53  40  10]
 ...
 [ 29 194 495 ...  47   4   0]
 [ 42 251 409 ... 107  17   1]
 [ 36 174 397 ... 138  15   3]]
[[   0    0    0 ...    0 1008   14]
 [   0   34   54 ...   12  878   22]
 [   0    0    0 ...   34  551  388]
 ...
 [   0    6   56 ...   44  592  211]
 [   0    0    0 ...    0  571  453]
 [   0    0    0 ...    0  728  296]]
[[  0   0   0 ...   1 554 469]
 [  0   0  53 ...  72 143 809]
 [  0   0 172 ...   0  50 974]
 ...
 [  0   0   4 ... 575 248  25]
 [  0  10  52 ... 429 177  59]
 [  0   1  39 ... 458 227  56]]
[[425 599   0 ...   0   0   0]
 [450 574   0 ...   0   0   0]
 [423 601   0 ...   0   0   0]
 ...
 [631 392   1 ...   0   0   0]
 [831  99  25 ...  26  15   1]
 [678 190  60 ..

[[ 18 264 652 ...  36   0   0]
 [  1 223 509 ... 139   0   0]
 [  5 148 470 ... 175   0   0]
 ...
 [  0  43 391 ... 236  35   5]
 [  2  28 457 ... 102   6   0]
 [240 138 503 ...   3   0   0]]
[[   0  451  573 ...    0 1024    0]
 [   0  194  830 ...    0 1024    0]
 [   0  100  924 ...    0 1024    0]
 ...
 [   0   19  222 ...  254  612  116]
 [   1   11   51 ...  316  614    0]
 [   0    5   22 ...  192  797    0]]
[[   0    0    0 ...   24   61  858]
 [   0    0    0 ...   43   36  893]
 [   0    0    0 ...    7   42  975]
 ...
 [   0    0    0 ...   20   30  908]
 [   0    0    0 ...    0    0 1024]
 [   0    0    0 ...    6    2 1016]]
[[  4 551 211 ...  12   0   0]
 [  0 560 184 ... 136   0   0]
 [  0 511 235 ... 162   2   0]
 ...
 [142 703 175 ...   0   0   0]
 [ 45 560 314 ...   1   0   0]
 [ 44 691 278 ...   0   0   0]]
[[514 171 111 ...  10   8   3]
 [593 171 241 ...   1   0   0]
 [808 216   0 ...   0   0   0]
 ...
 [375 284 179 ...  35   9  11]
 [375 230 145 ...  74  58  64]


...

*** Iteration 90 ***
[[  0   0  19 ...  46  25 637]
 [  0   0  41 ...  57  25 538]
 [  0   0  32 ... 240  40 376]
 ...
 [  6   8  54 ... 154 204 432]
 [  0   2  26 ... 140 172 472]
 [  0   0  30 ... 174 228 444]]
[[  1  52 334 ... 160  18   0]
 [  0 136 376 ...  64   1   0]
 [  1 101 315 ... 110  57   7]
 ...
 [ 23 147 527 ... 115  11   0]
 [ 53 232 545 ...  47   0   0]
 [ 29 194 495 ...  47   4   0]]
[[   0    0    0 ...    0 1008   14]
 [   0   34   54 ...   12  878   22]
 [   0    0    0 ...   34  551  388]
 ...
 [   0    6   56 ...   44  592  211]
 [   0    0    0 ...    0  571  453]
 [   0    0    0 ...    0  728  296]]
[[137  84 200 ... 385   4   0]
 [158  48  79 ... 409  27  28]
 [  0  10 381 ... 486 182 183]
 ...
 [  0   4  21 ... 439 187 103]
 [  0   1  39 ... 458 227  56]
 [  0   0  26 ... 414 210  34]]
[[425 599   0 ...   0   0   0]
 [423 601   0 ...   0   0   0]
 [470 554   0 ...   0   0   0]
 ...
 [773 202  44 ...   0   0   0]
 [831  99  25 ...  26  15   1]
 [678 190  60 .

*** Iteration 92 ***
[[  0   0  19 ...  46  25 637]
 [  0   0  41 ...  57  25 538]
 [  0   0  32 ... 240  40 376]
 ...
 [  6   8  54 ... 154 204 432]
 [  0   2  26 ... 140 172 472]
 [  0   0  30 ... 174 228 444]]
[[  1  52 334 ... 160  18   0]
 [  0 136 376 ...  64   1   0]
 [  1 101 315 ... 110  57   7]
 ...
 [ 23 147 527 ... 115  11   0]
 [ 53 232 545 ...  47   0   0]
 [ 29 194 495 ...  47   4   0]]
[[   0    0    0 ...    0 1008   14]
 [   0   34   54 ...   12  878   22]
 [   0    0    0 ...   34  551  388]
 ...
 [   0    6   56 ...   44  592  211]
 [   0    0    0 ...    0  571  453]
 [   0    0    0 ...    0  728  296]]
[[137  84 200 ... 385   4   0]
 [158  48  79 ... 409  27  28]
 [  0  10 381 ... 486 182 183]
 ...
 [  0   4  21 ... 439 187 103]
 [  0   1  39 ... 458 227  56]
 [  0   0  26 ... 414 210  34]]
[[425 599   0 ...   0   0   0]
 [423 601   0 ...   0   0   0]
 [470 554   0 ...   0   0   0]
 ...
 [773 202  44 ...   0   0   0]
 [831  99  25 ...  26  15   1]
 [678 190  60 .

*** Iteration 94 ***
[[  0   0  19 ...  46  25 637]
 [  0   0  41 ...  57  25 538]
 [  0   0  32 ... 240  40 376]
 ...
 [  6   8  54 ... 154 204 432]
 [  0   2  26 ... 140 172 472]
 [  0   0  30 ... 174 228 444]]
[[  1  52 334 ... 160  18   0]
 [  0 136 376 ...  64   1   0]
 [  1 101 315 ... 110  57   7]
 ...
 [ 23 147 527 ... 115  11   0]
 [ 53 232 545 ...  47   0   0]
 [ 29 194 495 ...  47   4   0]]
[[   0    0    0 ...    0 1008   14]
 [   0   34   54 ...   12  878   22]
 [   0    0    0 ...   34  551  388]
 ...
 [   0    6   56 ...   44  592  211]
 [   0    0    0 ...    0  571  453]
 [   0    0    0 ...    0  728  296]]
[[137  84 200 ... 385   4   0]
 [158  48  79 ... 409  27  28]
 [  0  10 381 ... 486 182 183]
 ...
 [  0   4  21 ... 439 187 103]
 [  0   1  39 ... 458 227  56]
 [  0   0  26 ... 414 210  34]]
[[425 599   0 ...   0   0   0]
 [423 601   0 ...   0   0   0]
 [470 554   0 ...   0   0   0]
 ...
 [773 202  44 ...   0   0   0]
 [831  99  25 ...  26  15   1]
 [678 190  60 .

*** Iteration 96 ***
[[  0   0  19 ...  46  25 637]
 [  0   0  41 ...  57  25 538]
 [  0   0  32 ... 240  40 376]
 ...
 [  6   8  54 ... 154 204 432]
 [  0   2  26 ... 140 172 472]
 [  0   0  30 ... 174 228 444]]
[[  1  52 334 ... 160  18   0]
 [  0 136 376 ...  64   1   0]
 [  1 101 315 ... 110  57   7]
 ...
 [ 23 147 527 ... 115  11   0]
 [ 53 232 545 ...  47   0   0]
 [ 29 194 495 ...  47   4   0]]
[[   0    0    0 ...    0 1008   14]
 [   0   34   54 ...   12  878   22]
 [   0    0    0 ...   34  551  388]
 ...
 [   0    6   56 ...   44  592  211]
 [   0    0    0 ...    0  571  453]
 [   0    0    0 ...    0  728  296]]
[[137  84 200 ... 385   4   0]
 [158  48  79 ... 409  27  28]
 [  0  10 381 ... 486 182 183]
 ...
 [  0   4  21 ... 439 187 103]
 [  0   1  39 ... 458 227  56]
 [  0   0  26 ... 414 210  34]]
[[425 599   0 ...   0   0   0]
 [423 601   0 ...   0   0   0]
 [470 554   0 ...   0   0   0]
 ...
 [773 202  44 ...   0   0   0]
 [831  99  25 ...  26  15   1]
 [678 190  60 .

*** Iteration 98 ***
[[  0   0  19 ...  46  25 637]
 [  0   0  41 ...  57  25 538]
 [  0   0  32 ... 240  40 376]
 ...
 [  6   8  54 ... 154 204 432]
 [  0   2  26 ... 140 172 472]
 [  0   0  30 ... 174 228 444]]
[[  1  52 334 ... 160  18   0]
 [  0 136 376 ...  64   1   0]
 [  1 101 315 ... 110  57   7]
 ...
 [ 23 147 527 ... 115  11   0]
 [ 53 232 545 ...  47   0   0]
 [ 29 194 495 ...  47   4   0]]
[[   0    0    0 ...    0 1008   14]
 [   0   34   54 ...   12  878   22]
 [   0    0    0 ...   34  551  388]
 ...
 [   0    6   56 ...   44  592  211]
 [   0    0    0 ...    0  571  453]
 [   0    0    0 ...    0  728  296]]
[[137  84 200 ... 385   4   0]
 [158  48  79 ... 409  27  28]
 [  0  10 381 ... 486 182 183]
 ...
 [  0   4  21 ... 439 187 103]
 [  0   1  39 ... 458 227  56]
 [  0   0  26 ... 414 210  34]]
[[425 599   0 ...   0   0   0]
 [423 601   0 ...   0   0   0]
 [470 554   0 ...   0   0   0]
 ...
 [773 202  44 ...   0   0   0]
 [831  99  25 ...  26  15   1]
 [678 190  60 .

In [178]:
def BoVW_image_feature_vector(img_col_hist, centroids):
    distances = cdist(img_col_hist, centroids ,'euclidean')
    points = np.array([np.argmin(i) for i in distances])
    feature = np.bincount(points, minlength=len(centroids))
    return feature

In [186]:
list(train_imgs_hist.values())[0][0]

array([[  0,   0,   0, ...,  24,  61, 858],
       [  0,   0,   0, ...,  43,  36, 893],
       [  0,   0,   1, ...,  47,  26, 862],
       ...,
       [  0,   0, 256, ..., 442,  67,   6],
       [  0,   0, 263, ..., 239,  92,  36],
       [  0,   0, 324, ..., 268,  16,   0]])

In [187]:
BoVW_image_feature_vector(list(train_imgs_hist.values())[0][0], centroids)

array([ 6,  3,  4,  0,  0,  0,  0,  0,  1,  0,  0,  0, 20,  0,  7,  4,  0,
        4,  0,  1, 14,  3,  3,  0,  3, 11,  0, 11,  0,  6,  2,  1],
      dtype=int64)

In [194]:
img_BoVW_all = {}
for img_type, img_hist_array in train_imgs_hist.items():
    img_BoVW = np.empty((len(img_hist_array), 32), dtype=np.int32)
    for i in range(len(img_hist_array)):
        img_BoVW[i] = BoVW_image_feature_vector(img_hist_array[i], centroids)
    img_BoVW_all[img_type] = img_BoVW

In [196]:
img_BoVW_all["batters_box"].shape

(49, 32)